In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import codecs
import gc
import json
import time

DATA_DIR = 'data'

CHUNKSIZE = 10000000
# MAX_ROW_NUM = 50000000
# MAX_ROW_NUM = 3000000

NYC_RANGE = (-74.3400, -73.6700, 40.4500, 40.9500)
NYC_RANGE_EXTENDED = (-74.4667, -73.0083, 40.4203, 41.3077)
NYC_RANGE_FILENAME = 'gnyc.png'
nyc_map = plt.imread(os.path.join(DATA_DIR, NYC_RANGE_FILENAME))

NYC_EXTENDED_FILENAME = 'gnyce.png'
nyc_map_extended = plt.imread(os.path.join(DATA_DIR, NYC_EXTENDED_FILENAME))

WASHED_DATA_DIR = 'washed'
WASHED_FILE_NAME = 'train_w.csv'

INPUT_FILE_PATH = os.path.join(WASHED_DATA_DIR, WASHED_FILE_NAME)

COLUMN_NAMES = ['fare_amount', 'pickup_longitude', 'pickup_latitude', 
                'dropoff_longitude', 'dropoff_latitude', 'passenger_count']



In [2]:
def lookup(s):
    """
    This is an extremely fast approach to datetime parsing.
    For large data, the same dates are often repeated. Rather than
    re-parse these, we store all unique dates, parse them, and
    use a lookup to convert all dates.
    """
    dates = {date:pd.to_datetime(date) for date in s.unique()}
    return s.map(dates)

In [7]:
sum_of_x = pd.DataFrame(columns=COLUMN_NAMES)
sum_of_xsqure = pd.DataFrame(columns=COLUMN_NAMES)
st = time.time()
for trunk in pd.read_csv(INPUT_FILE_PATH, chunksize=10000, parse_dates=['pickup_datetime'], infer_datetime_format=True):
    # print (time.time() - st)
    # print (trunk.pickup_datetime)
    trunk.pickup_datetime.date
    break

0      2009-06-15 17:26:21
1      2010-01-05 16:52:16
2      2011-08-18 00:35:00
3      2012-04-21 04:30:42
4      2010-03-09 07:51:00
5      2011-01-06 09:50:45
6      2012-11-20 20:35:00
7      2012-01-04 17:22:00
8      2012-12-03 13:10:00
9      2009-09-02 01:11:00
10     2012-04-08 07:30:50
11     2009-11-06 01:04:03
12     2013-07-02 19:54:00
13     2011-04-05 17:11:05
14     2014-02-19 07:22:00
15     2009-07-22 16:08:00
16     2010-07-07 14:52:00
17     2014-12-06 20:36:22
18     2010-09-07 13:18:00
19     2013-02-12 12:15:46
20     2009-08-06 18:17:23
21     2010-12-06 12:29:00
22     2009-12-10 15:37:00
23     2011-06-21 16:15:00
24     2011-06-28 19:47:00
25     2012-05-04 06:11:20
26     2013-08-11 00:52:00
27     2014-02-19 16:03:00
28     2015-06-21 21:46:34
29     2009-01-09 16:10:00
               ...        
9970   2012-11-24 21:22:58
9971   2013-12-04 07:51:41
9972   2014-12-31 07:32:54
9973   2009-06-28 23:28:00
9974   2010-01-04 09:12:01
9975   2011-09-24 21:48:21
9

In [4]:
df = pd.read_csv(INPUT_FILE_PATH, nrows=10)
df.pickup_datetime

0    2009-06-15 17:26:21 UTC
1    2010-01-05 16:52:16 UTC
2    2011-08-18 00:35:00 UTC
3    2012-04-21 04:30:42 UTC
4    2010-03-09 07:51:00 UTC
5    2011-01-06 09:50:45 UTC
6    2012-11-20 20:35:00 UTC
7    2012-01-04 17:22:00 UTC
8    2012-12-03 13:10:00 UTC
9    2009-09-02 01:11:00 UTC
Name: pickup_datetime, dtype: object

In [22]:
st = time.time()
df = pd.read_csv(INPUT_FILE_PATH, nrows=100000, parse_dates=['pickup_datetime'], infer_datetime_format=True)
print (time.time() - st)

1.8500699996948242


In [3]:
st = time.time()
df = pd.read_csv(INPUT_FILE_PATH, nrows=100000)
print (time.time() - st)

0.6449751853942871


In [6]:
st = time.time()
df = pd.read_csv(INPUT_FILE_PATH, nrows=100000, parse_dates=['pickup_datetime'])
print (time.time() - st)

28.47322177886963


In [7]:
st = time.time()
df = pd.read_csv(INPUT_FILE_PATH, nrows=100000, parse_dates=['pickup_datetime'], infer_datetime_format=True)
new_years, new_months, new_days, new_hours = zip(*[(d.year, d.month, d.day, d.hour) for d in df.pickup_datetime])
df = df.assign(pickup_year=new_years, pickup_month=new_months, pickup_day=new_days, pickup_hour=new_hours)
df.drop(columns=['pickup_datetime'])
print (time.time() - st)

2.45011043548584


In [24]:
def parsetuple(d):
    a = d.split(' ')
    date = a[0]
    time = a[1]
    y, m, day = date.split('-')
    h = time.split(':')[0]
    return int(y), int(m), int(day), int(h)
st = time.time()
df = pd.read_csv(INPUT_FILE_PATH, nrows=100000)
new_years, new_months, new_days, new_hours = zip(*[parsetuple(d) for d in df.pickup_datetime])
df = df.assign(pickup_year=new_years, pickup_month=new_months, pickup_day=new_days, pickup_hour=new_hours)
df.drop(columns=['pickup_datetime'])
print (time.time() - st)

1.361006259918213


In [19]:
def parsetuple(d):
    a = d.split(' ')
    date = a[0]
    time = a[1]
    y, m, day = date.split('-')
    h = time.split(':')[0]
    return int(d[0:4]), int(d[5:7]), int(d[8:10]), int(d[11:13])
st = time.time()
df = pd.read_csv(INPUT_FILE_PATH, nrows=100000)
new_years, new_months, new_days, new_hours = zip(*[parsetuple(d) for d in df.pickup_datetime])
df = df.assign(pickup_year=new_years, pickup_month=new_months, pickup_day=new_days, pickup_hour=new_hours)
df.drop(columns=['pickup_datetime'])
print (time.time() - st)

1.5310471057891846


In [23]:
st = time.time()
df = pd.read_csv(INPUT_FILE_PATH, nrows=100000)
new_years, new_months, new_days, new_hours = zip(*[(int(d[0:4]), int(d[5:7]), int(d[8:10]), int(d[11:13])) for d in df.pickup_datetime])
df = df.assign(pickup_year=new_years, pickup_month=new_months, pickup_day=new_days, pickup_hour=new_hours)
df.drop(columns=['pickup_datetime'])
print (time.time() - st) 

1.1640467643737793


In [25]:
df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_year,pickup_month,pickup_day,pickup_hour
0,2009-06-15 17:26:21.0000001,4.5,2009-06-15 17:26:21 UTC,-73.844311,40.721319,-73.841610,40.712278,1,2009,6,15,17
1,2010-01-05 16:52:16.0000002,16.9,2010-01-05 16:52:16 UTC,-74.016048,40.711303,-73.979268,40.782004,1,2010,1,5,16
2,2011-08-18 00:35:00.00000049,5.7,2011-08-18 00:35:00 UTC,-73.982738,40.761270,-73.991242,40.750562,2,2011,8,18,0
3,2012-04-21 04:30:42.0000001,7.7,2012-04-21 04:30:42 UTC,-73.987130,40.733143,-73.991567,40.758092,1,2012,4,21,4
4,2010-03-09 07:51:00.000000135,5.3,2010-03-09 07:51:00 UTC,-73.968095,40.768008,-73.956655,40.783762,1,2010,3,9,7


In [ ]:
df = pd.read_csv('washed\\train_wf.csv', nrows=1000)
df.head()

NameError: name 'pd' is not defined